In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import os
import torch

custom_path = os.path.expanduser("/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj") 
os.environ["FINN_BUILD_DIR"] = custom_path

# Verify the environment variable has been set correctly
print(os.environ["FINN_BUILD_DIR"])

build_dir = os.environ["FINN_BUILD_DIR"]
 
#build_dir = os.environ["FINN_BUILD_DIR"]
PATH = './model.pth'

/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj


In [2]:
import os

file_path = "/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj"
if os.path.exists(file_path):
    print("The file exists at the specified path")
else:
    print("The file does not exist at the specified path")

The file exists at the specified path


In [3]:
from torch import nn
import brevitas.nn as qnn


class AI85KWS20Netv3_brevitas(nn.Module):
    """
    Compound KWS20 v3 Audio net, all with Conv1Ds, but with brevitas
    """

    # num_classes = n keywords + 1 unknown
    def __init__(
            self,
            num_classes=21,
            num_channels=128,
            dimensions=(128, 1),  # pylint: disable=unused-argument
            bias=False,
            weight_quant_type='INT',
            weight_bit_width=8,
            weight_scaling_impl_type='CONST',
            weight_scaling_const=1,
            bias_quant_type='INT',
            bias_bit_width=8,
            act_bit_width=8,
            act_scaling_impl_type='CONST',
            act_scaling_const=1,
            **kwargs

    ):
        super().__init__()
        self.indentity = qnn.QuantIdentity(bit_width=8,
                                           return_quant_tensor=False)
        self.voice_conv1 = qnn.QuantConv1d(num_channels, 100, 1, bias=False,stride=1, padding=0,
                                           weight_bit_width=4,
                                           )
        self.voice_conv1_relu = qnn.QuantReLU(bit_width=8,
                                              return_quant_tensor=False)

        # T: 128 F: 100
        self.voice_conv2 = qnn.QuantConv1d(100, 96, 3, bias=False,stride=1, padding=0,
                                           weight_bit_width=4,
                                           )
        self.voice_conv2_relu = qnn.QuantReLU(bit_width=8,
                                              return_quant_tensor=False)
        # T: 126 F : 96
        self.voice_conv3 = qnn.QuantConv1d(96, 64, 3, bias=False,stride=1, padding=0,
                                           weight_bit_width=4,
                                           )
        self.voice_conv3_maxpool = qnn.QuantMaxPool1d(kernel_size=2, stride=2)
        self.voice_conv3_relu = qnn.QuantReLU(bit_width=8,
                                              return_quant_tensor=False)
        # T: 62 F : 64
        self.voice_conv4 = qnn.QuantConv1d(64, 48, 3, bias=False,stride=1, padding=0,
                                           weight_bit_width=4,
                                           )
        self.voice_conv4_relu = qnn.QuantReLU(bit_width=8,
                                              return_quant_tensor=False)
        # T : 60 F : 48
        self.kws_conv1 = qnn.QuantConv1d(48, 64, 3, bias=False,stride=1, padding=0,
                                         weight_bit_width=4,
                                         )
        self.kws_conv1_maxpool = qnn.QuantMaxPool1d(kernel_size=2, stride=2)
        self.kws_conv1_relu = qnn.QuantReLU(bit_width=8,
                                            return_quant_tensor=False)

        # T: 30 F : 64
        self.kws_conv2 = qnn.QuantConv1d(64, 96, 3, bias=False,stride=1, padding=0,
                                         weight_bit_width=4,
                                         )
        self.kws_conv2_relu = qnn.QuantReLU(bit_width=8,
                                            return_quant_tensor=False)
        # T: 28 F : 96
        self.kws_conv3 = qnn.QuantConv1d(96, 100, 3, bias=False,stride=1, padding=0,
                                         weight_bit_width=4,
                                         )
        self.kws_conv3_relu = qnn.QuantReLU(bit_width=8,
                                            return_quant_tensor=False)

        # T : 14 F: 100
        self.kws_conv4 = qnn.QuantConv1d(100, 64, 6, bias=False,stride=1, padding=0,
                                         weight_bit_width=4,
                                         )
        self.kws_conv4_maxpool = qnn.QuantMaxPool1d(kernel_size=2, stride=2)
        self.kws_conv4_relu = qnn.QuantReLU(bit_width=8,
                                            return_quant_tensor=False)
        # T : 2 F: 128
        self.fc = nn.Linear(640, num_classes, bias=False)
                                

    def forward(self, x):  # pylint: disable=arguments-differ
        """Forward prop"""
        # Run CNN
        x = self.indentity(x)
        x = self.voice_conv1(x)
        x = self.voice_conv1_relu(x)
        x = self.voice_conv2(x)
        x = self.voice_conv2_relu(x)
        x = self.voice_conv3(x)
        x = self.voice_conv3_maxpool(x)
        x = self.voice_conv3_relu(x)
        x = self.voice_conv4(x)
        x = self.voice_conv4_relu(x)
        x = self.kws_conv1(x)
        x = self.kws_conv1_maxpool(x)
        x = self.kws_conv1_relu(x)
        x = self.kws_conv2(x)
        x = self.kws_conv2_relu(x)
        x = self.kws_conv3(x)
        x = self.kws_conv3_relu(x)
        x = self.kws_conv4(x)
        x = self.kws_conv4_maxpool(x)
        x = self.kws_conv4_relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


def samet_brevitas(pretrained=False, **kwargs):
    """
    Constructs a AI85KWS20Net model.
    rn AI85KWS20Net(**kwargs)
    """
    assert not pretrained
    return AI85KWS20Netv3_brevitas(**kwargs)

doruk_qnn = samet_brevitas()

In [4]:
import torch
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

doruk_qnn.load_state_dict(torch.load(PATH,map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
bo.export_finn_onnx(doruk_qnn, (1,128,128), build_dir + "/doruk_qnn.onnx");

In [6]:
showInNetron(build_dir+"/doruk_qnn.onnx")

Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/doruk_qnn.onnx' at http://0.0.0.0:8081


In [7]:
from qonnx.core.modelwrapper import ModelWrapper
model = ModelWrapper(build_dir+"/doruk_qnn.onnx")

/home/kuantek/FINN/finn/deps/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(


In [8]:
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())

In [9]:
from qonnx.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
model = model.transform(Change3DTo4DTensors())
model.save(build_dir + "/doruk_tidy.onnx")
showInNetron(build_dir+"/doruk_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/doruk_tidy.onnx' at http://0.0.0.0:8081


In [10]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

model = ModelWrapper(build_dir+"/doruk_tidy.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = build_dir+"/doruk_lenet_preproc.onnx"
bo.export_finn_onnx(totensor_pyt, ishape, chkpt_preproc_name)

# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
model = model.transform(MergeONNXModels(pre_model))
model.save(build_dir + "/doruk_lenet_preproc.onnx")
showInNetron(build_dir+"/doruk_lenet_preproc.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/doruk_lenet_preproc.onnx' at http://0.0.0.0:8081


/home/kuantek/FINN/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:119: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [11]:
from qonnx.core.modelwrapper import ModelWrapper
model = ModelWrapper(build_dir+"/doruk_lenet_preproc.onnx")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(build_dir + "/doruk_lenet_tidy.onnx")
showInNetron(build_dir+"/doruk_lenet_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/doruk_lenet_tidy.onnx' at http://0.0.0.0:8081


In [12]:
from finn.transformation.streamline import Streamline
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

model = ModelWrapper(build_dir + "/doruk_lenet_tidy.onnx")
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(Streamline())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model.save(build_dir + "/end2end_cnv_w1a1_streamlined.onnx")

In [13]:
showInNetron(build_dir+"/end2end_cnv_w1a1_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/end2end_cnv_w1a1_streamlined.onnx' at http://0.0.0.0:8081


In [14]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.infer_data_layouts import InferDataLayouts

# choose the memory mode for the MVTU units, decoupled or const
mem_mode = "decoupled"

model = ModelWrapper(build_dir + "/end2end_cnv_w1a1_streamlined.onnx")
model = model.transform(to_hls.InferBinaryMatrixVectorActivation(mem_mode))
model = model.transform(to_hls.InferQuantizedMatrixVectorActivation(mem_mode))
# TopK to LabelSelect
model = model.transform(to_hls.InferLabelSelectLayer())
# input quantization (if any) to standalone thresholding
model = model.transform(to_hls.InferThresholdingLayer())
model = model.transform(to_hls.InferConvInpGen())
model = model.transform(to_hls.InferStreamingMaxPool())
# get rid of Reshape(-1, 1) operation between hlslib nodes
model = model.transform(RemoveCNVtoFCFlatten())
# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())
# infer tensor data layouts
model = model.transform(InferDataLayouts())
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(build_dir + "/end2end_cnv_w1a1_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model.save(build_dir + "/end2end_cnv_w1a1_dataflow_model.onnx")

/home/kuantek/FINN/finn/src/finn/transformation/move_reshape.py:76: UserWarning: Could not absorb transpose->flatten                                         into subsequent node
  warnings.warn(


In [15]:
showInNetron(build_dir+"/end2end_cnv_w1a1_dataflow_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/end2end_cnv_w1a1_dataflow_model.onnx' at http://0.0.0.0:8081


In [16]:
model = ModelWrapper(build_dir + "/end2end_cnv_w1a1_dataflow_model.onnx")
fc_layers = model.get_nodes_by_op_type("MatrixVectorActivation")
# each tuple is (PE, SIMD, in_fifo_depth) for a layer
folding = [
    (1, 1, [32]),
    (1, 1, [32]),
    (1, 1, [32]),
    (1, 1, [32]),
    (1, 1, [32]),
    (1, 1, [32]),
    (1, 1, [32]),
    (1, 1, [32]),
    
]
for fcl, (pe, simd, ififodepth) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepths", ififodepth)

# use same SIMD values for the sliding window operators
swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator")
for i in range(len(swg_layers)):
    swg_inst = getCustomOp(swg_layers[i])
    simd = folding[i][1]
    swg_inst.set_nodeattr("SIMD", simd)

model = model.transform(GiveUniqueNodeNames())
model.save(build_dir + "/end2end_cnv_w1a1_folded.onnx")

In [17]:
showInNetron(build_dir + "/end2end_cnv_w1a1_folded.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/kuantek/FINN/finn/notebooks/samet_workspace/vivado_proj/end2end_cnv_w1a1_folded.onnx' at http://0.0.0.0:8081


In [22]:
test_pynq_board = "ZCU104"
target_clk_ns = 20

from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(build_dir+"/end2end_cnv_w1a1_folded.onnx")
model = model.transform(ZynqBuild(platform = test_pynq_board, period_ns = target_clk_ns))

/home/kuantek/FINN/finn/src/finn/transformation/fpgadataflow/floorplan.py:108: UserWarning: 37 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/home/kuantek/FINN/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:292: UserWarning: Output FIFO for Thresholding_Batch_2_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
